# A simple example doing Bayesian inference for a harmonic oscillator

Uses MCMC through the KissMCMC samplers.

The forward model and other bits are defined in `bayesian-example-base.jl`

This example assumes that the times of measurements are errror-free, a simplification which the next example lifts.

In [1]:
# This is where the forward model, probabilistic model etc are defined.
using NBInclude
nbinclude("bayesian-ex-base.ipynb")
using Plots,StatPlots # this works only up to StatPlots 9e665438 & on 0.5 of PyCall too (14 July 2016)
@time 1 # initialze
# choose to use the analytic or ODE-forward model
fwd! = [fwd_ana!, fwd_ode!][1];

  0.000002 seconds (148 allocations: 10.151 KB)


    +(AbstractArray, DataArrays.DataArray) at /home/mauro/.julia/v0.4/DataArrays/src/operators.jl:276
is ambiguous with: 
    +(WoodburyMatrices.SymWoodbury, AbstractArray{T<:Any, 2}) at /home/mauro/.julia/v0.4/WoodburyMatrices/src/SymWoodburyMatrices.jl:107.
To fix, define 
    +(WoodburyMatrices.SymWoodbury, DataArrays.DataArray{T<:Any, 2})
before the new definition.
    +(AbstractArray, DataArrays.AbstractDataArray) at /home/mauro/.julia/v0.4/DataArrays/src/operators.jl:300
is ambiguous with: 
    +(WoodburyMatrices.SymWoodbury, AbstractArray{T<:Any, 2}) at /home/mauro/.julia/v0.4/WoodburyMatrices/src/SymWoodburyMatrices.jl:107.
To fix, define 
    +(WoodburyMatrices.SymWoodbury, DataArrays.AbstractDataArray{T<:Any, 2})
before the new definition.


## Make the synthetic `xv_measured`

In [2]:
sigma = 0.4
const ts_measured, xv_measured, A, ω, ϕ, ts_true, xv_true = make_synthetic_measurements(theta_true = [2.5, 1.1, 3], # [A, ω, ϕ]
                                                                               sigma_x=sigma,
                                                                               sigma_v=sigma,
                                                                               sigma_t=0)
@assert all(ts_measured.==ts_true)

[Plots.jl] Initializing backend: pyplot



In [3]:
## Plot the measurements
using Plots, StatPlots # needs current master of StatPlots & on 0.5 of PyCall too (14 July 2016)
plotmeasurements(ts_measured,xv_measured,A,ω,ϕ,sigma_x=sigma,sigma_v=sigma)

## Probabilistic model setup

Fitting parameters `[A,ω,ϕ,sigma] = theta`

In [4]:
varnames = ["A","ω", "ϕ", "σ"]

# Likelilhood
const fwdout = init_fwd(ts_measured) # Note, this will be modified in place
function loglikelihood(theta)
    # Closure over fwdout, fwd!, xv_measured, ts_measured
    # make sure those are `const`!
    A,ω,ϕ,sigma = theta
    fwd!(fwdout, ts_measured, A,ω,ϕ)
    loglikelihood(fwdout, xv_measured, sigma)
end

# Normal & uniform priors

logprior_A(A) = 0<=A ? 0.0 : -Inf # amplitude is positive
ω_max = 15.0
logprior_ω(ω) = 0<=ω<ω_max ? 0.0 : -Inf # ω is bounded
logprior_ϕ(ϕ) = 0<=ϕ<2*pi ? 0.0 : -Inf # ϕ is bounded

sigma_est = 0.2 # our prior estimate of sigma
sigma_est_sigma = 0.2 # our estimate of the std of sigma
logprior_sigma(sigma) = sigma<=0 ? -Inf : -(sigma-sigma_est)^2/(2*sigma_est_sigma)

logprior = (theta) -> (logprior_A(theta[1]) +
                       logprior_ω(theta[2]) +
                       logprior_ϕ(theta[3]) +
                       logprior_sigma(theta[4]) )

# This is the pdf which will be estimated with MCMC
logposterior = @anon theta -> loglikelihood(theta) + logprior(theta);

## MCMC fitting

First define some parameters. Generally `_m` referes to variables connected to the Metropolis sampler, `_e` to the `emcee` sampler.

In [5]:
niter = 5*10^5 # number of MC steps
nthin = 10     # only store theta every so often
nburnin = niter÷3 # how much burn-in
nchains = 50 # number of chains used in the emcee sampler
niter_e = niter÷nchains # iterations in the emcee sampler reduced accordingly
nburnin_e = niter_e÷3; # burnin

3333

In [6]:
#################
# Metropolis MCMC
#################

# Using a Gaussian proposal distribution.  Note, this needs to be
# symmetric for Metropolis!

sig = 0.01  # this needs tuning, accept_ratio of 1/4 is good, they say.
const sigma_ppdf = [sig, sig, sig, sig]
sample_ppdf(theta) = [randn()*sigma_ppdf[i]+theta[i] for i=1:length(theta)]

theta_true = [A, ω, ϕ, sigma];  # good IC
theta0 = [2.1, 1.1, 1.1, 0.2]; # decent IC
# note that the emcee sampler does poorly with a poor IC!
#theta0 = [1.74,0.001, 0.25, 1.77] # bad IC

metropolis(logposterior, sample_ppdf, theta0, niter=2)
print("Metropolis: ")
@time thetas_m, accept_ratio_m = metropolis(logposterior, sample_ppdf, theta0,
                                            niter=niter, nthin=nthin, nburnin=nburnin)
print_results(thetas_m, accept_ratio_m, names=varnames, title="Metropolis", theta_true=theta_true)

Metropolis:   1.467071 seconds (16.07 M allocations: 307.230 MB, 2.61% gc time)
Metropolis
Ratio of accepted/total steps: 0.7101735796528407

var 	 err	median	 mean 	 mode 	 std
A 	 0.11 	 2.61 	 2.61 	 2.59 	 0.16
ω 	 0.01 	 1.09 	 1.09 	 1.09 	 0.02
ϕ 	 0.04 	 3.04 	 3.04 	 3.03 	 0.12
σ 	 0.16 	 0.56 	 0.57 	 0.53 	 0.09



In [7]:
#################
# emcee MCMC
#################

emcee(logposterior, (theta0, 0.1), niter=10, nchains=2)
print("emcee:")
@time thetas_ec, accept_ratio_ec = emcee(logposterior, (theta0, 0.1),
                                         niter=niter_e, nthin=nthin, nchains=nchains, nburnin=nburnin_e)
# When running this problem with IC far from the maximum, then emcee produces
thetas_e, accept_ratio_e = squash_chains(thetas_ec, accept_ratio_ec, drop_low_accept_ratio=true)
print_results(thetas_e, accept_ratio_e, names=varnames, title="emcee", theta_true=theta_true)

emcee:  1.707372 seconds (19.77 M allocations: 440.791 MB, 3.17% gc time)
Dropping low accept-ratio chain 9
Dropping low accept-ratio chain 46
emcee
Ratio of accepted/total steps: 0.5662560621968903

var 	 err	median	 mean 	 mode 	 std
A 	 0.1 	 2.6 	 2.6 	 2.56 	 0.17
ω 	 0.01 	 1.09 	 1.09 	 1.09 	 0.02
ϕ 	 0.04 	 3.04 	 3.04 	 3.02 	 0.12
σ 	 0.16 	 0.56 	 0.57 	 0.57 	 0.09



Output evaluation: plot the 1D and 2D distributions in a corner-plot.  Note that all the parameters are well estimated.  The std-deviation of the parameters depends on how much error the synthetic data contains.  $\omega$ and $\phi$ are anti-correlated, the rest are un-correlated.


In [8]:
cornerplot(thetas_e', label=["A","ω", "ϕ", "σ"], size=(600,600))